# **Diplomado de Machine Learning con Python**
## Tarea 06 – Sesión 09 (PCA NUEVOS)

## **Aplicación de Análisis de Componentes Principales (PCA)**

📊 En esta tarea se aborda la aplicación del Análisis de Componentes Principales (PCA) como parte del preprocesamiento en flujos de machine learning y como técnica de reducción de dimensionalidad en un conjunto de datos multivariado.

El proceso se divide en dos etapas principales:

1. Preprocesamiento del conjunto original.

2. Aplicación del flujo a datos nuevos no vistos.

En esta segunda etapa aplicamos todo el flujo de preprocesamiento y reducción dimensional (PCA) a un conjunto de datos nuevo y nunca antes visto por la computadora, utilizando los artefactos previamente entrenados y guardados.



![PCA](https://raw.githubusercontent.com/josue-rdgzcb/ml-python-scidata/refs/heads/main/tasks/tarea_06/PCA.png)

## Aplicación de PCA a datos nuevos

In [1]:
#####################################################
#
# APLICAR PCA A DATOS QUE NUNCA HA VISTO
#
#####################################################
# Deben cargarse los archivos
# - preprocessor_cat.joblib
# - pca_pipe_num.joblib
# - pca_metadata.json
# - T_train_final_objetivo.csv
# - csv de tus nuevos datos
# Devolverá T_new_final.csv: el csv de PCA aplicado a los nuevos datos
#####################################################

# === Cargar artefactos para inferencia ===
import joblib
import json
import pandas as pd

preprocessor_cat = joblib.load("preprocessor_cat.joblib")
pca_pipe = joblib.load("pca_pipe_num.joblib")
with open("pca_metadata.json", "r") as f:
    meta = json.load(f)

cols_num = meta["cols_num"]
cols_cat = meta["cols_cat"]
pc_cols  = meta["pc_cols"]
cat_out_cols = meta["cat_out_cols"]

# PCA DE ENTRENAMIENTO CON OBJETIVO
entrenamiento = pd.read_csv("T_train_final_objetivo.csv")   ### todas las pca, categoricas y objetivo
entrenamiento_pca_objetivo = entrenamiento[pc_cols]

# NUEVOS DATOS
new_df = pd.read_csv("palmer_penguins_desconocido.csv")
X_new_num = new_df[cols_num]
X_new_cat = new_df[cols_cat]

# Categóricas (mismo encoder, sin re-ajustar)
X_new_cat_proc = preprocessor_cat.transform(X_new_cat)
df_new_cat_encode = pd.DataFrame(X_new_cat_proc, columns=cat_out_cols, index=new_df.index)

# Numéricas → (preproc_num + StdScaler + PCA) con el pipeline guardado
T_new = pca_pipe.transform(X_new_num)
T_new_df = pd.DataFrame(T_new, columns=pc_cols, index=new_df.index)

# Final
T_new_final = pd.concat([T_new_df, df_new_cat_encode], axis=1)
T_new_final.to_csv("T_new_final.csv",index=False)

In [2]:
T_new_df

,PC1,PC2,PC3
0,-0.841364,0.475004,-1.362379
1,-2.361071,-0.670436,-0.486969
2,-1.709069,-0.207261,0.381230
3,-2.126665,-0.495623,-1.026534
4,1.171109,-1.624328,0.460535
5,1.900155,-0.810802,0.115131
6,2.476134,0.236090,-0.252484
7,1.488393,-0.897623,0.168613
8,-1.403709,-0.215924,0.542793
9,-0.454950,0.044006,0.727617


## Visualización comparativa en 2D entre los datos de entrenamiento y los nuevos datos transformados

In [6]:
import plotly.express as px
import plotly.graph_objects as go

# --- 1) figura base: train coloreado por objetivo ---
y_train = entrenamiento["objetivo"]
df_plot2 = entrenamiento_pca_objetivo.iloc[:, :2].copy()
df_plot2["objetivo"] = y_train.loc[entrenamiento_pca_objetivo.index].astype(str)

fig = px.scatter(
    df_plot2, x="PC1", y="PC2", color="objetivo",
    opacity=0.85, title="PC1 vs PC2 — Train (objetivo) + Nuevos",
    height=550
)
fig.update_traces(marker=dict(size=6))

# --- 2) overlay: nuevos con número de fila como texto ---
new2 = T_new_df.loc[:, ["PC1", "PC2"]].copy()

fig.add_trace(
    go.Scatter(
        x=new2["PC1"],
        y=new2["PC2"],
        mode="markers+text",  # muestra puntos y texto
        name="nuevos",
        text=new2.index.astype(str),  # usa el índice como etiqueta
        textposition="top center",    # posición del texto
        marker=dict(size=10, line=dict(width=1)),
        opacity=0.95,
        showlegend=True
    )
)

fig.update_layout(xaxis_title="PC1", yaxis_title="PC2")
fig.show()


## Visualización comparativa en 3D entre los datos de entrenamiento y los nuevos datos transformados

In [4]:
import plotly.express as px
import plotly.graph_objects as go

# --- base: entrenamiento 3D ---
df3 = entrenamiento_pca_objetivo.iloc[:, :3].copy()
df3["objetivo"] = y_train.loc[entrenamiento_pca_objetivo.index].astype(str)

fig3 = px.scatter_3d(
    df3, x="PC1", y="PC2", z="PC3",
    color="objetivo", opacity=0.85,
    title="PC1–PC2–PC3 — Train (objetivo) + Nuevos",
    height=600
)
fig3.update_traces(marker=dict(size=5))

# --- overlay: nuevos con índice como texto ---
new3 = T_new_df.loc[:, ["PC1", "PC2", "PC3"]].copy()

fig3.add_trace(
    go.Scatter3d(
        x=new3["PC1"],
        y=new3["PC2"],
        z=new3["PC3"],
        mode="markers+text",  # muestra puntos y texto
        name="nuevos",
        text=new3.index.astype(str),  # usa el índice como etiqueta
        textposition="top center",    # posición del texto
        marker=dict(size=6, line=dict(width=1)),
        opacity=0.95,
        showlegend=True
    )
)

fig3.update_layout(
    scene=dict(
        xaxis_title="PC1",
        yaxis_title="PC2",
        zaxis_title="PC3"
    )
)
fig3.show()


## Conjeturas y conclusiones

Al observar el gráfico PCA en dos dimensiones, hice conjeturas sobre la especie de cada uno de los nuevos puntos basándome en su cercanía visual con los grupos conocidos del conjunto de entrenamiento. Supuse que los puntos 0 y 9 podrían corresponder a la especie Chinstrap, los puntos 1, 2, 3 y 8 a Adelie, y los puntos 4, 5, 6 y 7 a Gentoo.

Al comparar estas conjeturas con las especies correctas de mi conjunto de datos desconocido, observé lo siguiente:

- Los puntos 4, 5, 6 y 7 efectivamente pertenecen a Gentoo, lo que confirma que ese grupo estaba bien definido en el espacio PCA.

- El punto 9 también fue correctamente identificado como Chinstrap.

- Los puntos 1, 2 y 3 fueron acertadamente clasificados como Adelie.

- Sin embargo, el punto 0, que había supuesto como Chinstrap, en realidad pertenece a Adelie.

- El punto 8, que había clasificado como Adelie, resultó ser Chinstrap.

En resumen, la mayoría de las conjeturas fueron correctas, especialmente en los casos donde los grupos estaban bien separados visualmente. Los errores ocurrieron en zonas donde las clases se superponen ligeramente en el plano PCA, lo que sugiere que aunque el espacio reducido conserva bastante bien la estructura de las especies, hay casos en donde se comparten límites y la distinción no es tan evidente visualmente.

Al incorporar una tercera componente principal (PC3), capturamos más varianza explicada del conjunto original, lo que permitió una representación más precisa de la estructura interna de los datos. Mi observación sugiere que usar tres componentes mejora la distinción entre especies, al menos en la interpretación visual del gráfico.

<div align="center">
  <img src="https://allisonhorst.github.io/palmerpenguins/reference/figures/lter_penguins.png" alt="Palmer Penguins" width="450"/>
</div>
